In [1]:
import uproot # type: ignore
import pandas as pd # type: ignore
import numpy as np # type: ignore
import awkward as ak # type: ignore
import matplotlib # type: ignore
from mpl_toolkits.mplot3d import Axes3D # type: ignore
import matplotlib.pyplot as plt # type: ignore
import os
import math
import csv
import seaborn as sns # type: ignore
import sympy as smp # type: ignore
import sympy.stats # type: ignore
from scipy import stats,signal,ndimage,special,fft # type: ignore
from scipy.optimize import curve_fit # type: ignore
from scipy.integrate import quad # type: ignore
from mpl_toolkits.axes_grid1 import make_axes_locatable # type: ignore

In [2]:
tracks = uproot.open("/nashome/z/zaemoore/2x2_CAF/2p2h_truth_sample.root")
tracks["TruthTree"].show()
tracks["TruthTree"].keys()

branches = tracks["TruthTree"].arrays()[0]
file_index = branches['file_index']
print("File indeces: ", np.unique(file_index))

name                 | typename                 | interpretation                
---------------------+--------------------------+-------------------------------
true_energy          | std::vector<double>      | AsJagged(AsDtype('>f8'), he...
true_p_x             | std::vector<double>      | AsJagged(AsDtype('>f8'), he...
true_p_y             | std::vector<double>      | AsJagged(AsDtype('>f8'), he...
true_p_z             | std::vector<double>      | AsJagged(AsDtype('>f8'), he...
true_p_mag           | std::vector<double>      | AsJagged(AsDtype('>f8'), he...
true_length          | std::vector<double>      | AsJagged(AsDtype('>f8'), he...
true_angle           | std::vector<double>      | AsJagged(AsDtype('>f8'), he...
true_angle_rot       | std::vector<double>      | AsJagged(AsDtype('>f8'), he...
true_angle_incl      | std::vector<double>      | AsJagged(AsDtype('>f8'), he...
true_angle_x         | std::vector<double>      | AsJagged(AsDtype('>f8'), he...
true_angle_y         | std::

In [4]:
file_idx = 2

true_energy = branches['true_energy'][file_index == file_idx]
true_p_x = branches['true_p_x'][file_index == file_idx]
true_p_y = branches['true_p_y'][file_index == file_idx]
true_p_z = branches['true_p_z'][file_index == file_idx]
true_p_mag = branches['true_p_mag'][file_index == file_idx]
true_length = branches['true_length'][file_index == file_idx]
true_angle = branches['true_angle'][file_index == file_idx]
true_angle_rot = branches['true_angle_rot'][file_index == file_idx]
true_angle_incl = branches['true_angle_incl'][file_index == file_idx]
true_angle_x = branches['true_angle_x'][file_index == file_idx]
true_angle_y = branches['true_angle_y'][file_index == file_idx]
true_angle_z = branches['true_angle_z'][file_index == file_idx]
true_track_start_x = branches['true_track_start_x'][file_index == file_idx]
true_track_start_y = branches['true_track_start_y'][file_index == file_idx]
true_track_start_z = branches['true_track_start_z'][file_index == file_idx]
true_track_end_x = branches['true_track_end_x'][file_index == file_idx]
true_track_end_y = branches['true_track_end_y'][file_index == file_idx]
true_track_end_z = branches['true_track_end_z'][file_index == file_idx]
true_pdg = branches['true_pdg'][file_index == file_idx]
true_nproton = branches['true_nproton'][file_index == file_idx]
spill_index = branches['spill_index'][file_index == file_idx]
interaction_id = branches['interaction_id'][file_index == file_idx]
event = branches['event'][file_index == file_idx]
run = branches['run'][file_index == file_idx]
subrun = branches['subrun'][file_index == file_idx]
caf_file_name = branches['caf_file_name'][file_index == file_idx]

In [ ]:
for ev in event:
    true_start_x_ev = true_track_start_x[event == ev]
    true_start_y_ev = true_track_start_y[event == ev]
    true_start_z_ev = true_track_start_z[event == ev]
    true_end_x_ev = true_track_end_x[event == ev]
    true_end_y_ev = true_track_end_y[event == ev]
    true_end_z_ev = true_track_end_z[event == ev]


    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(222, projection='3d')
    ax1 = fig.add_subplot(221)
    ax2 = fig.add_subplot(223)
    ax3 = fig.add_subplot(224)

    def make_cathode_or_anode(y1, y2, z1, z2, x):
        y = np.linspace(y1, y2, 100)
        z = np.linspace(z1, z2, 100)
        Y, Z = np.meshgrid(y, z)
        X = np.full(Y.shape, x)
        return X, Y, Z

    #Make alpha list from linspace or something similar that is based on the length of true start z all
    alphas = np.linspace(0.5, 1.0, len(true_start_z_ev))

    # Plot TPC Surfaces
    anode_y1s = [-62.076, -62.076, -62.076, -62.076, -62.076, -62.076, -62.076, -62.076]
    anode_y2s = [62.076, 62.076, 62.076, 62.076, 62.076, 62.076, 62.076,62.076]
    anode_z1s = [2.462, 2.462, 2.462, 2.462, -2.462, -2.462, -2.462, -2.462]
    anode_z2s = [2.462+62.076, 2.462+62.076, 2.462+62.076, 2.462+62.076, -2.462-62.076, -2.462-62.076, -2.462-62.076, -2.462-62.076]
    anode_xs = [33.5+30.27225, -33.5+30.27225, 33.5-30.27225, -33.5-30.27225, 33.5+30.27225, -33.5+30.27225, 33.5-30.27225, -33.5-30.27225]

    cathode_y1s = [-62.076, -62.076, -62.076, -62.076]
    cathode_y2s = [62.076, 62.076, 62.076, 62.076]
    cathode_z1s = [2.462, 2.462, -2.462, -2.462]
    cathode_z2s = [2.462+62.076, 2.462+62.076, -2.462-62.076, -2.462-62.076]
    cathode_xs = [33.5, -33.5, 33.5, -33.5]


    for i in range(len(cathode_y1s)):
        X_cathode, Y_cathode, Z_cathode = make_cathode_or_anode(cathode_y1s[i], cathode_y2s[i], cathode_z1s[i], cathode_z2s[i], cathode_xs[i])
        ax.plot_surface(Z_cathode,X_cathode,Y_cathode, color='b', alpha=0.15)
    for i in range(len(anode_y1s)):
        X_anode, Y_anode, Z_anode = make_cathode_or_anode(anode_y1s[i], anode_y2s[i], anode_z1s[i], anode_z2s[i], anode_xs[i])
        ax.plot_surface(Z_anode,X_anode,Y_anode, color='g', alpha=0.15)

    ax.set_xlabel('Beam Direction [Z, cm]')
    ax.set_ylabel('Drift Direction [X, cm]')
    ax.set_zlabel('Vertical Direction [Y, cm]')

    # Plot outlines of detector for projections:
    ax1.plot([anode_z1s[0], anode_z1s[0]], [anode_xs[0], cathode_xs[0]], color='k', alpha=0.3)
    ax1.plot([anode_z2s[0], anode_z2s[0]], [anode_xs[0], cathode_xs[0]], color='k', alpha=0.3)
    ax1.plot([anode_z1s[0], anode_z2s[0]], [anode_xs[0], anode_xs[0]], color='k', alpha=0.3)
    ax1.plot([anode_z1s[0], anode_z2s[0]], [cathode_xs[0], cathode_xs[0]], color='k', alpha=0.3)

    ax1.plot([anode_z1s[0], anode_z1s[0]], [anode_xs[2], cathode_xs[0]], color='k', alpha=0.3)
    ax1.plot([anode_z2s[0], anode_z2s[0]], [anode_xs[2], cathode_xs[0]], color='k', alpha=0.3)
    ax1.plot([anode_z1s[0], anode_z2s[0]], [anode_xs[2], anode_xs[2]], color='k', alpha=0.3)
    #ax1.plot([anode_z1s[0], anode_z2s[0]], [cathode_xs[0], cathode_xs[0]], color='k', alpha=0.3)

    ax1.plot([anode_z1s[0], anode_z1s[0]], [anode_xs[1], cathode_xs[1]], color='k', alpha=0.3)
    ax1.plot([anode_z2s[0], anode_z2s[0]], [anode_xs[1], cathode_xs[1]], color='k', alpha=0.3)
    ax1.plot([anode_z1s[0], anode_z2s[0]], [anode_xs[1], anode_xs[1]], color='k', alpha=0.3)
    ax1.plot([anode_z1s[0], anode_z2s[0]], [cathode_xs[1], cathode_xs[1]], color='k', alpha=0.3)

    ax1.plot([anode_z1s[0], anode_z1s[0]], [anode_xs[3], cathode_xs[1]], color='k', alpha=0.3)
    ax1.plot([anode_z2s[0], anode_z2s[0]], [anode_xs[3], cathode_xs[1]], color='k', alpha=0.3)
    ax1.plot([anode_z1s[0], anode_z2s[0]], [anode_xs[3], anode_xs[3]], color='k', alpha=0.3)
    #ax1.plot([anode_z1s[0], anode_z2s[0]], [cathode_xs[1], cathode_xs[1]], color='k', alpha=0.3)

    ax1.plot([anode_z1s[-1], anode_z1s[-1]], [anode_xs[0], cathode_xs[0]], color='k', alpha=0.3)
    ax1.plot([anode_z2s[-1], anode_z2s[-1]], [anode_xs[0], cathode_xs[0]], color='k', alpha=0.3)
    ax1.plot([anode_z1s[-1], anode_z2s[-1]], [anode_xs[0], anode_xs[0]], color='k', alpha=0.3)
    ax1.plot([anode_z1s[-1], anode_z2s[-1]], [cathode_xs[0], cathode_xs[0]], color='k', alpha=0.3)

    ax1.plot([anode_z1s[-1], anode_z1s[-1]], [anode_xs[2], cathode_xs[0]], color='k', alpha=0.3)
    ax1.plot([anode_z2s[-1], anode_z2s[-1]], [anode_xs[2], cathode_xs[0]], color='k', alpha=0.3)
    ax1.plot([anode_z1s[-1], anode_z2s[-1]], [anode_xs[2], anode_xs[2]], color='k', alpha=0.3)
    #ax1.plot([anode_z1s[-1], anode_z2s[-1]], [cathode_xs[0], cathode_xs[0]], color='k', alpha=0.3)

    ax1.plot([anode_z1s[-1], anode_z1s[-1]], [anode_xs[1], cathode_xs[1]], color='k', alpha=0.3)
    ax1.plot([anode_z2s[-1], anode_z2s[-1]], [anode_xs[1], cathode_xs[1]], color='k', alpha=0.3)
    ax1.plot([anode_z1s[-1], anode_z2s[-1]], [anode_xs[1], anode_xs[1]], color='k', alpha=0.3)
    ax1.plot([anode_z1s[-1], anode_z2s[-1]], [cathode_xs[1], cathode_xs[1]], color='k', alpha=0.3)

    ax1.plot([anode_z1s[-1], anode_z1s[-1]], [anode_xs[3], cathode_xs[1]], color='k', alpha=0.3)
    ax1.plot([anode_z2s[-1], anode_z2s[-1]], [anode_xs[3], cathode_xs[1]], color='k', alpha=0.3)
    ax1.plot([anode_z1s[-1], anode_z2s[-1]], [anode_xs[3], anode_xs[3]], color='k', alpha=0.3)
    #ax1.plot([anode_z1s[-1], anode_z2s[-1]], [cathode_xs[1], cathode_xs[1]], color='k', alpha=0.3)

    ax1.set_xlabel('Beam Direction [Z, cm]')
    ax1.set_ylabel('Drift Direction [X, cm]')

    ax2.plot([anode_z1s[-1], anode_z1s[-1]], [anode_y1s[0], anode_y2s[0]], color='k', alpha=0.3)
    ax2.plot([anode_z1s[0], anode_z1s[0]], [anode_y1s[0], anode_y2s[0]], color='k', alpha=0.3)
    ax2.plot([anode_z2s[-1], anode_z2s[-1]], [anode_y1s[0], anode_y2s[0]], color='k', alpha=0.3)
    ax2.plot([anode_z2s[0], anode_z2s[0]], [anode_y1s[0], anode_y2s[0]], color='k', alpha=0.3)
    ax2.plot([anode_z1s[-1], anode_z2s[-1]], [anode_y1s[0], anode_y1s[0]], color='k', alpha=0.3)
    ax2.plot([anode_z1s[0], anode_z2s[0]], [anode_y1s[0], anode_y1s[0]], color='k', alpha=0.3)
    ax2.plot([anode_z1s[-1], anode_z2s[-1]], [anode_y2s[0], anode_y2s[0]], color='k', alpha=0.3)
    ax2.plot([anode_z1s[0], anode_z2s[0]], [anode_y2s[0], anode_y2s[0]], color='k', alpha=0.3)

    ax2.set_xlabel('Beam Direction [Z, cm]')
    ax2.set_ylabel('Vertical Direction [Y, cm]')


    ax3.plot([anode_xs[0], cathode_xs[0]], [anode_y1s[0], anode_y1s[0]], color='k', alpha=0.3)
    ax3.plot([anode_xs[1], cathode_xs[1]], [anode_y1s[0], anode_y1s[0]], color='k', alpha=0.3)
    ax3.plot([anode_xs[2], cathode_xs[0]], [anode_y2s[0], anode_y2s[0]], color='k', alpha=0.3)
    ax3.plot([anode_xs[3], cathode_xs[1]], [anode_y2s[0], anode_y2s[0]], color='k', alpha=0.3)
    ax3.plot([anode_xs[2], cathode_xs[0]], [anode_y1s[0], anode_y1s[0]], color='k', alpha=0.3)
    ax3.plot([anode_xs[3], cathode_xs[1]], [anode_y1s[0], anode_y1s[0]], color='k', alpha=0.3)
    ax3.plot([anode_xs[0], cathode_xs[0]], [anode_y2s[0], anode_y2s[0]], color='k', alpha=0.3)
    ax3.plot([anode_xs[1], cathode_xs[1]], [anode_y2s[0], anode_y2s[0]], color='k', alpha=0.3)
    ax3.plot([anode_xs[0], anode_xs[0]], [anode_y1s[0], anode_y2s[0]], color='k', alpha=0.3)
    ax3.plot([anode_xs[1], anode_xs[1]], [anode_y1s[0], anode_y2s[0]], color='k', alpha=0.3)
    ax3.plot([anode_xs[2], anode_xs[2]], [anode_y1s[0], anode_y2s[0]], color='k', alpha=0.3)
    ax3.plot([anode_xs[3], anode_xs[3]], [anode_y1s[0], anode_y2s[0]], color='k', alpha=0.3)
    ax3.plot([cathode_xs[0], cathode_xs[0]], [anode_y1s[0], anode_y2s[0]], color='k', alpha=0.3)
    ax3.plot([cathode_xs[1], cathode_xs[1]], [anode_y1s[0], anode_y2s[0]], color='k', alpha=0.3)

    ax3.set_xlabel('Drift Direction [X, cm]')
    ax3.set_ylabel('Vertical Direction [Y, cm]')

    for i in range(len(true_start_z_ev)):

        true_start_x, true_end_x = true_start_x_ev[i], true_end_x_ev[i]
        true_start_y, true_end_y = true_start_y_ev[i], true_end_y_ev[i]
        true_start_z, true_end_z = true_start_z_ev[i], true_end_z_ev[i]

        ax.plot([true_start_z, true_end_z], [true_start_x, true_end_x], [true_start_y, true_end_y],
                color='blue', alpha=alphas[i])
        if i==0:
            ax.scatter(true_start_z, true_start_x, true_start_y, color='red', marker='*', s=50, label='True Match Start')
            ax.scatter(true_end_z, true_end_x, true_end_y, color='red', marker='s', s=25, label='True Match End')
        else:
            ax.scatter(true_start_z, true_start_x, true_start_y, color='red', alpha=alphas[i], marker='*', s=50)
            ax.scatter(true_end_z, true_end_x, true_end_y, color='red', alpha=alphas[i], marker='s', s=25)
        ax1.plot([true_start_z, true_end_z], [true_start_x, true_end_x], alpha=alphas[i], color='blue')
        ax1.scatter(true_start_z, true_start_x, color='red', alpha=alphas[i], marker='*', s=50)
        ax1.scatter(true_end_z, true_end_x, color='red', alpha=alphas[i], marker='s', s=25)

        ax2.plot([true_start_z, true_end_z], [true_start_y, true_end_y],  color='blue', alpha=alphas[i])
        ax2.scatter(true_start_z, true_start_y, color='red', alpha=alphas[i], marker='*', s=50)
        ax2.scatter(true_end_z, true_end_y, color='red', alpha=alphas[i], marker='s', s=25)

        ax3.plot([true_start_x, true_end_x], [true_start_y, true_end_y], color='blue', alpha=alphas[i])
        ax3.scatter(true_start_x, true_start_y, color='red', alpha=alphas[i], marker='*', s=50)
        ax3.scatter(true_end_x, true_end_y, color='red', alpha=alphas[i], marker='s', s=25)

    #set bound limits for all plots
    ax1.set_xlim(-75, 75)
    ax1.set_ylim(-75, 75)
    ax2.set_xlim(-75, 75)
    ax2.set_ylim(-75, 75)
    ax3.set_xlim(-75, 75)
    ax3.set_ylim(-75, 75)

    ax.set_xlim3d(-75, 75)
    ax.set_ylim3d(-75, 75)
    ax.set_zlim3d(-75, 75)

    ax.legend(loc='upper left')
    plt.tight_layout()
    plt.savefig('/nashome/z/zaemoore/2x2_CAF/evd_outputs/track_'+str(file_idx)+'_'+str(ev)+'_true.png')